In [ ]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload 2

In [ ]:
from src.models.qwen3 import Qwen3

t = Qwen3(model_name = "Qwen/Qwen3-8B", device = "cuda:7")

t.model

In [ ]:
from src.datasets.lener import LenerDataset

l = LenerDataset(tokenizer=t.tokenizer)

In [ ]:
data = l.load_dataset()
data

In [ ]:
# all = trainable params: 21,823,488 || all params: 8,212,558,848 || trainable%: 0.2657
# full_att = trainable params: 7,667,712 || all params: 8,198,403,072 || trainable%: 0.0935
# ffn = trainable params: 9,437,184 || all params: 8,200,172,544 || trainable%: 0.1151
## full_att_plus_ffn: trainable params: 17,104,896 || all params: 8,207,840,256 || trainable%: 0.2084
# low_rank = trainable params: 10,911,744 || all params: 8,201,647,104 || trainable%: 0.1330
# high_rank = trainable params: 87,293,952 || all params: 8,278,029,312 || trainable%: 1.0545
# high XX = trainable params: 349,175,808 || all params: 8,539,911,168 || trainable%: 4.0888
# baseline = trainable params: 1,179,648 || all params: 8,191,915,008 || trainable%: 0.0144
#  0.6B_all = trainable params: 5,046,272 || all params: 601,096,192 || trainable%: 0.8395
# 1.7B = trainable params: 8,716,288 || all params: 1,729,291,264 || trainable%: 0.5040
# 4B = trainable params: 16,515,072 || all params: 4,038,983,168 || trainable%: 0.4089

In [ ]:
from src.peft_configs.lora import LoraAdapter
# PRESET = 'baseline'
# PRESET = 'full_attention'
PRESET = 'all'
# PRESET = 'ffn'
# PRESET = 'full_attention_plus_ffn'
# PRESET = 'low_rank'
# PRESET = 'high_rank'
# PRESET = 'high_rank_XX'

lora = LoraAdapter(
    model=t.model,
    lora_preset = PRESET
)

model = lora.apply_lora()
model

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"

# Training args
training_args = TrainingArguments(
    output_dir=f'./outputs/checkpoints/Qwen3_8b_full_{PRESET}',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    # gradient_accumulation_steps=4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="no",
    learning_rate=2e-4,
    warmup_steps=10,
    fp16=torch.cuda.is_available(),
    report_to="none",
    # device=model.device
    
)
# model.to('cuda:7')
# torch.cuda.set_device(7)
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    tokenizer=t.tokenizer,
    data_collator=DataCollatorForLanguageModeling(t.tokenizer, mlm=False),
    
)

# Train
trainer.train()